Dependencies:
* matplotlib
* numpy
* scikit-learn
* pandas

In [1]:
%pylab inline
import pandas as pd
import json
import os
import glob

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


# Dataset

In [2]:
#dataset = "dataset/UC6_O3_lateral"
dataset = "dataset/UC4_O2_spherical_c_left_top_corner"
selected_features = [
    "finger_angle_index", "finger_angle_mrl", "finger_angle_thumb",
    "finger_tangential_mrl", "finger_normal_index", "finger_tangential_index",
    "finger_tangential_thumb", "finger_normal_thumb", "finger_normal_mrl"
]

In [3]:
def load_metadata(filename):
    with open(filename, "r") as f:
        return json.load(f)


def load_measurements(metadata):
    path = os.path.join(metadata["path"], metadata["sensor_filename"])
    return pd.read_csv(path, index_col=0)

For each grasp we stored some metadata. The most important for you is the label.

In [4]:
metadata = load_metadata(list(glob.glob(os.path.join(dataset, "*_failure_1.json")))[0])

In [5]:
metadata

{'label': 'failure',
 'sensor_filename': 'UC4_O2_failure_1.csv',
 'path': 'dataset/UC4_O2_spherical_c_left_top_corner',
 'p': [-0.0576, 0.1588, 0.4308],
 'rpy': [-0.8179, 0.3213, -3.123],
 'fraction': 0.9455,
 'grasp_typ': 'spherical',
 'object': 'UC4_O2',
 'orientation': 'c',
 'grasp_location': 'left top corner',
 'hand_calibration': 'MNS9'}

For the metadata we can also load the corresponding sensor measurements. Note that each sample has a different length!

In [6]:
measurements = load_measurements(metadata)[selected_features]

In [7]:
measurements

finger_angle_index  finger_angle_mrl  finger_angle_thumb  \
0              0.000000         -0.276250           -0.057961   
1              0.000000         -0.276250           -0.057961   
2              0.000000         -0.276250           -0.057961   
3              0.000000         -0.276250           -0.057961   
4              0.000000         -0.276250           -0.057961   
..                  ...               ...                 ...   
557            0.720113          0.937497            0.809848   
558            0.720113          0.937497            0.809848   
559            0.720113          0.937497            0.809848   
560            0.720113          0.937497            0.809848   
561            0.720113          0.937497            0.809848   

     finger_tangential_mrl  finger_normal_index  finger_tangential_index  \
0                     -1.0                 -1.0                     -1.0   
1                     -1.0                 -1.0                     -1.0   
2                     -1.0                 -1.0                     -1.0   
3                     -1.0                 -1.0                     -1.0   
4                     -1.0                 -1.0                     -1.0   
..                     ...                  ...                      ...   
557                   -5.0                 -7.0                     -4.0   
558                   -5.0                 -7.0                     -4.0   
559                   -5.0                 -7.0                     -4.0   
560                   -5.0                 -7.0                     -4.0   
561                   -5.0                 -7.0                     -4.0   

     finger_tangential_thumb  finger_normal_thumb  finger_normal_mrl  
0                        0.0                  2.0                0.0  
1                        0.0                  2.0                0.0  
2                        0.0                  2.0                0.0  
3                        0.0                  2.0                0.0  
4                        0.0                  2.0                0.0  
..                       ...                  ...                ...  
557                      8.0                -67.0               -9.0  
558                      8.0                -67.0               -9.0  
559                      8.0                -67.0               -9.0  
560                      8.0                -67.0               -9.0  
561                      8.0                -67.0               -9.0  

[562 rows x 9 columns]

In [8]:
def load_dataset(dataset, selected_features):
    metadata_filenames = list(sorted(glob.glob(os.path.join(dataset, "*.json"))))
    y = []
    X = []
    for fn in metadata_filenames:
        metadata = load_metadata(fn)
        measurements = load_measurements(metadata)[selected_features]
        X.append(measurements.to_numpy())
        y.append(metadata["label"] == "success_and_stable")
    return X, np.array(y)

In [9]:
X_raw, y_raw = load_dataset(dataset, selected_features)

Ratio positive:negative

In [10]:
np.count_nonzero(y_raw), len(y_raw) - np.count_nonzero(y_raw)

(45, 91)

# Preprocessing

Before applying machine learning, make sure to convert each sample to a vector of the same length!

In [11]:
def preprocess(X, y, n_steps):
    X_new = []
    y_new = []
    for i in range(len(X)):
        if n_steps > len(X[i]):
            continue  # not enough measurements
        #indices = np.arange(len(X[i]) - n_steps, len(X[i]))
        indices = np.linspace(0, len(X[i]) - 1, n_steps).astype(int)
        X_new.append(X[i][indices])
        y_new.append(y[i])
    return np.array(X_new).reshape(len(X_new), -1), np.array(y_new)

In [16]:
n_history = 2

In [17]:
X_train, y_train = preprocess(X_raw, y_raw, n_history)

In [25]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=3).fit(X_train, y_train)

# Machine Learning

Now it is time to do machine learning. :)

**Try to obtain the best classifier possible.**

You can use any ML library, but we recommend to use [sklearn](https://scikit-learn.org/stable/modules/classes.html). In the end, we will test the classifier's performance on a test set that you don't have yet. So make sure to not overfit the dataset that you have! How would you do that?

In [18]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

model = Pipeline([
    ["scaler", StandardScaler()],
    #["classifier", KNeighborsClassifier(n_neighbors=3)]
    ["features", PolynomialFeatures(degree=2, include_bias=False)],
    ["classifier", LogisticRegression(C=1.0, fit_intercept=True)]
    #["classifier", RandomForestClassifier()]
    #["classifier", GaussianNB()]
])

In [19]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV, HalvingGridSearchCV, RandomizedSearchCV, StratifiedKFold, LeaveOneOut

#params = {"classifier__n_neighbors": np.arange(2, 10)}  # KNeighborsClassifier
params = {"classifier__C": np.logspace(-5, 5, 11),  # LogisticRegression
          "features__degree": [1, 2, 3, 5]
         }
#params = {"classifier__max_depth": [2, 5, 10, None]}  # RandomForestClassifier
grid = RandomizedSearchCV(
    model,
    scoring="balanced_accuracy",
    n_jobs=4,
    cv=StratifiedKFold(n_splits=10),
    #cv=LeaveOneOut(),
    param_distributions=params  # RandomizedSearchCV
    #param_grid=params  # GridSearchCV
)
grid.fit(X_train, y_train)

/home/afabisch/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/afabisch/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
                   estimator=Pipeline(steps=[['scaler', StandardScaler()],
                                             ['features',
                                              PolynomialFeatures(include_bias=False)],
                                             ['classifier',
                                              LogisticRegression()]]),
                   n_jobs=4,
                   param_distributions={'classifier__C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05]),
                                        'features__degree': [1, 2, 3, 5]},
                   scoring='balanced_accuracy')

In [21]:
grid.best_params_

{'features__degree': 1, 'classifier__C': 1.0}

In [22]:
for mts, sts, p in zip(grid.cv_results_["mean_test_score"], grid.cv_results_["std_test_score"], grid.cv_results_["params"]):
    print(f"{mts:.3f} +- {sts:.3f} - {p}")

0.500 +- 0.000 - {'features__degree': 1, 'classifier__C': 0.0001}
0.719 +- 0.124 - {'features__degree': 2, 'classifier__C': 100.0}
0.718 +- 0.171 - {'features__degree': 5, 'classifier__C': 0.001}
0.827 +- 0.164 - {'features__degree': 1, 'classifier__C': 1.0}
0.500 +- 0.000 - {'features__degree': 3, 'classifier__C': 0.0001}
0.751 +- 0.124 - {'features__degree': 2, 'classifier__C': 100000.0}
0.802 +- 0.198 - {'features__degree': 1, 'classifier__C': 0.1}
0.751 +- 0.131 - {'features__degree': 5, 'classifier__C': 0.1}
0.753 +- 0.137 - {'features__degree': 5, 'classifier__C': 10.0}
0.757 +- 0.156 - {'features__degree': 3, 'classifier__C': 10000.0}


In [126]:
model.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('features', PolynomialFeatures(include_bias=False)),
                ['classifier', LogisticRegression()]])

# Evaluation on Test Set

In [23]:
X_test_raw, y_test_raw = load_dataset(dataset + "_test", selected_features)
X_test, y_test = preprocess(X_test_raw, y_test_raw, n_history)
grid.score(X_test, y_test)

0.7916666666666667

In [24]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

cm = confusion_matrix(grid.predict(X_test), y_test)
ConfusionMatrixDisplay(cm, display_labels=["Stable", "Unstable"]).plot()